# Working Notebook for creating wndchrm features

## Step 1
### Load nessecary modules

In [1]:
# Add python modules

import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
from copy import deepcopy

# Add parent directory for custom modules
from sys import path as sysPath
sysPath.append('../')
sysPath.append('../Machine_Score/')

# Load custom modules that simr has loaded
import main_SIMR as simr
gm = simr.gm
im = simr.im
ic = simr.ic
fe = simr.fe
ms = simr.ms
sa = simr.sa
dc = simr.ms.dc

simr.test()
gm.test()
im.test()
ic.test()
fe.test()
ms.test()
dc.test()
sa.test()

SIMR: Hi!  You're in Matthew's main program for all things galaxy collisions
GM: Hi!  You're in Matthew's module for generally useful functions and classes
IM: Hi!  You're in Matthew's information module for SPAM
IC: Hi!  You're in Matthew's main code for all things image creation.
FE: Hi!  You're in Matthew's module for extracting feature values from images.
MS: Hi!  You're in Matthew's SIMR module for all things machine scoring images
DC: Hi!  You're in direct_image_compare.py
SA: Hi!  You're in Matthew's Main program for score analysis!


___
## Step 2
### Load Target Info class
Needed for loading the target image and running through models later.

In [2]:
tDir = '../targetDir'
tDir = gm.validPath(tDir)

tInfo = im.target_info_class( targetDir = tDir, printAll=False)
if tInfo.status == False:
    print("WARNING: target info class bad")
else:
    print("Target Good!: %s" % tInfo.get('target_id'))

# Get run info class
rInfo = tInfo.getRunInfo( rID='run_0000' )

if rInfo.status == False:
    print("WARNING")
else:
    print("Run Good!: %s"%rInfo.get('run_id'))

Target Good!: 587722984435351614
Run Good!: run_0000


In [3]:
testArg = gm.inArgClass()
testArg.setArg('newFeats',True)
testArg.setArg('printAll',True)
chime_0 = tInfo.readScoreParam('chime_0')
testArg.setArg('scoreParams',chime_0)

simr.simr_run( arg=testArg, rInfo=rInfo )

SIMR.pipelineRun: Inputs
	 - rDir: None
	 - rInfo: <class 'Support_Code.info_module.run_info_class'>
SIMR.pipelineRun: 
	 - rInfo:  <Support_Code.info_module.run_info_class object at 0x7f0cdc199df0>
FE: image_creator_run
FE: rInfo.status:  True
FE: given parameters: 1
FE: name/type: chime_0/wndchrm_all
FE: wndchrm_run_all: Notebook development
FE: wndchrm_run_all: Executing:
	 - $ wndchrm train -s /home/mbo2d/587722984435351614/zoo_merger_models/run_0000/wndchrm_files/ /home/mbo2d/587722984435351614/zoo_merger_models/run_0000/wndchrm_files/wndchrm_all.fit
FE: wndchrm_run_all: Execution Done
	 - Collecting wndchrm all data
FE: reorganize_wndchrm_run_data
	 - number of features: 1059
	 - number of images: 50
	 - feature labels head: [ Chebyshev-Fourier Coefficients () [0], Chebyshev-Fourier Coefficients () [1], ... ]
	 - feature labels tail: [ ..., Zernike Coefficients (Fourier ()) [70], Zernike Coefficients (Fourier ()) [71] ]
	 - Extracted feature values dataframe
	 -  head: [ run_id, 

In [4]:
imgLoc = rInfo.wndDir + 'chime_0.tiff'

img = gm.readImg( imgLoc, toType = None )
print(np.amax(img))

255


In [5]:
chimeLoc = './../param/chime_group_1.json'
chime_group_1 = gm.readJson( chimeLoc )
#gm.pprint(chime_group_1['chime_1'])

In [6]:
testArg = gm.inArgClass()
testArg.printAll = True

In [7]:
fe.target_collect_wndchrm_all_raw( testArg, tInfo = tInfo )

FE: target_collect_wndchrm_all_raw
FE: target_collect_wndchrm_all_raw: Loop: 1293 / 1293: Complete!
	 - Read 1290 of 1293 wndchrm run files.
	 - Final WNDCHRM DataFrame
	 - Shape: (63211, 1062)
	 - Unique Runs: 1290
	 - Unique Image Names: 50
	 - Header Length: 1062
	 - Headers head: [ run_id, image_name, zoo_merger_score, ... ]
	 - Headers tail: [ ... , Zernike Coefficients (Fourier ()) [70], Zernike Coefficients (Fourier ()) [71] ]


In [9]:
fe.wndchrm_target_all( gm.inArgClass(), tInfo )

	 - Collecting wndchrm all data


In [10]:
tInfo.printAll = True
tInfo.gatherRunInfos()
print("Done")

IM: Target.gatherRunInfos
	 - IM: gather_run_info LOOP: 1293 / 1293 COMPLETE
IM: Target.saveInfoFile():
	 - Saving target info file...
Done
